In [1]:
import random
import scipy
import numpy as np
from numpy import linalg
from scipy.stats import t, f, norm, ksone
import matplotlib.pyplot as plt

In [2]:
k = 5
Ig1 = [83, 85]
Ig2 = [84, 85, 85, 86, 86, 87]
Ig3 = [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90]
Ig4 = [89, 90, 90, 91]
Ig5 = [90, 92]
Ig = Ig1 + Ig2 + Ig3 + Ig4 + Ig5
n1 = len(Ig1)
n2 = len(Ig2)
n3 = len(Ig3)
n4 = len(Ig4)
n5 = len(Ig5)
n_list = [n1, n2, n3, n4, n5]
n = sum(n_list)

In [7]:
def get_Psi(n_list):
    Psi = []
    c1 = 0
    c2 = sum(n_list)
    for i in range(k):
        Psi.append([0]*c1 + [1]*n_list[i] + [0]*(c2-n_list[i]-c1))
        c1 += n_list[i]
    return np.matrix(Psi).T
def get_inv(A):
    return linalg.inv(A)
def get_F(Psi):
    return Psi.T @ Psi;
def get_beta(Psi, Y):
    return get_inv(get_F(Psi)) @ Psi.T @ Y
def get_e(Y, Psi, beta):
    return Y - Psi @ beta
def get_RSS(e):
    return (e.transpose() @ e)[0,0]
def get_TSS(Y):
    av_y = np.sum(Y)/Y.size
    return sum([(Y[i, 0]-av_y)**2 for i in range(Y.size)])
def get_R(RSS, TSS):
    return (TSS - RSS)/TSS

In [8]:
Psi= get_Psi(n_list)
print(f"Psi_t = \n{Psi.T}")

Psi_t = 
[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]


In [9]:
Y = np.matrix(Ig).T
print(f"Y_t = {Y.T}")

Y_t = [[83 85 84 85 85 86 86 87 86 87 87 87 88 88 88 88 88 89 90 89 90 90 91 90
  92]]


In [11]:
print(f"F: \n{get_F(Psi)}\n")
F_1 = get_inv(get_F(Psi))
print(f"F^-1: \n{F_1}\n")
beta = get_beta(Psi, Y)
print("beta = " + ",".join(map(str, beta.T[0].tolist())))
e = get_e(Y, Psi, beta)
print("\ne = " + ",".join(map(str, e.T[0].tolist())))

RSS = get_RSS(e)
print(f"\n RSS = {RSS}")
TSS = get_TSS(Y)
print(f"\n TSS = {TSS}")
R = get_R(RSS, TSS)
print(f"\n R^2 = {R}")

F: 
[[ 2  0  0  0  0]
 [ 0  6  0  0  0]
 [ 0  0 11  0  0]
 [ 0  0  0  4  0]
 [ 0  0  0  0  2]]

F^-1: 
[[0.5        0.         0.         0.         0.        ]
 [0.         0.16666667 0.         0.         0.        ]
 [0.         0.         0.09090909 0.         0.        ]
 [0.         0.         0.         0.25       0.        ]
 [0.         0.         0.         0.         0.5       ]]

beta = [84.0, 85.5, 87.81818181818181, 90.0, 91.0]

e = [-1.0, 1.0, -1.5, -0.5, -0.5, 0.5, 0.5, 1.5, -1.818181818181813, -0.818181818181813, -0.818181818181813, -0.818181818181813, 0.18181818181818699, 0.18181818181818699, 0.18181818181818699, 0.18181818181818699, 0.18181818181818699, 1.181818181818187, 2.181818181818187, -1.0, 0.0, 0.0, 1.0, -1.0, 1.0]

 RSS = 23.136363636363637

 TSS = 122.16

 R^2 = 0.8106060606060606


In [16]:
def check_significance_beta(Psi, beta, RSS, n, p, i):
    alpha = 0.05
    delta = beta[i]*(n-p)**0.5
    delta /= (RSS * get_inv(get_F(Psi))[i,i])**0.5
    delta = delta[0,0]
    p_value = 2.*(1-t.cdf(abs(delta), n-p))
    if(p_value < alpha):
        print(f"p_value ({p_value}) < alpha ({alpha}) -> beta{i+1} значим")
    else:
        print(f"p_value ({p_value}) > alpha ({alpha}) -> beta{i+1} не является значимым")
for i in range(k):
    check_significance_beta(Psi, beta, RSS, n, k, i)

p_value (0.0) < alpha (0.05) -> beta1 значим
p_value (0.0) < alpha (0.05) -> beta2 значим
p_value (0.0) < alpha (0.05) -> beta3 значим
p_value (0.0) < alpha (0.05) -> beta4 значим
p_value (0.0) < alpha (0.05) -> beta5 значим


In [19]:
def check_significance_R(R, n, p):
    alpha = 0.05
    delta= R/(1-R)*(n-p)*1./(p-1)
    p_value = 1-f.cdf(delta, p-1, n-p)
    if(p_value < alpha):
        print(f"p_value ({p_value}) <alpha ({alpha}) -> регрессия значима")
    else:
        print(f"p_value ({p_value}) >alpha ({alpha}) -> регрессия не является значимой")

In [20]:
check_significance_R(R, n, k)

p_value (5.407435041959729e-07) <alpha (0.05) -> регрессия значима


In [21]:
def groups_equality(beta, RSS, F_1, n, p, i, j):
    alpha = 0.05
    delta = (beta[i, 0] - beta[j, 0])/np.sqrt(RSS * (F_1[i,i] + F_1[j,j])) * np.sqrt(n-p)
    p_value = 2.*(1-t.cdf(abs(delta), n-p))  
    if(p_value < alpha):
        print(f"p_value ({p_value}) < alpha ({alpha}) -> beta{i+1} и beta{j+1} разные")
        return False
    else:
        print(f"p_value ({p_value}) > alpha ({alpha}) -> beta{i+1} и beta{j+1} одинаковы")
        return True

def check_dependence_age(β, RSS, F_1, n, p):
    check = True
    for i in range(p):
        for j in range(i+1, p):
            check = groups_equality(β, RSS, F_1, n, p, i, j) and check            
    if(check):
        print("Cодержание иммуноглобулина не зависит от возраста")
    else:
        print("Cодержание иммуноглобулина зависит от возраста")

In [22]:
check_dependence_age(beta, RSS, F_1, n, k)

p_value (0.10310040890082384) > alpha (0.05) -> beta1 и beta2 одинаковы
p_value (0.0001662014487755492) < alpha (0.05) -> beta1 и beta3 разные
p_value (2.782111500732043e-06) < alpha (0.05) -> beta1 и beta4 разные
p_value (2.4125702768884594e-06) < alpha (0.05) -> beta1 и beta5 разные
p_value (0.00039504010033408754) < alpha (0.05) -> beta2 и beta3 разные
p_value (2.5534724907849693e-06) < alpha (0.05) -> beta2 и beta4 разные
p_value (4.085183105129175e-06) < alpha (0.05) -> beta2 и beta5 разные
p_value (0.0023933387043622023) < alpha (0.05) -> beta3 и beta4 разные
p_value (0.0010025629060046448) < alpha (0.05) -> beta3 и beta5 разные
p_value (0.29579135227474707) > alpha (0.05) -> beta4 и beta5 одинаковы
Cодержание иммуноглобулина зависит от возраста
